# In this notebook We'll work with `Logistic Regression` and diferents penalities like None, L1, L2 and ElasticNet.
- The `Pima Indians Diabetes` dataset will be used

## 1 - Imports:

- `Libraries`

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

- `Dataset`

In [2]:
filename = '/home/joaocarvoli/Downloads/pima-indians-diabetes/diabetes.csv'
df = pd.read_csv(filename)

## 2 - Data Analisys and Preprocessing:

In [3]:
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [4]:
df.shape

(768, 9)

- `Spliting X and y`

In [5]:
X = df.iloc[:,:-1]
y = df.iloc[:,-1]

In [6]:
X

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
0,6,148,72,35,0,33.6,0.627,50
1,1,85,66,29,0,26.6,0.351,31
2,8,183,64,0,0,23.3,0.672,32
3,1,89,66,23,94,28.1,0.167,21
4,0,137,40,35,168,43.1,2.288,33
...,...,...,...,...,...,...,...,...
763,10,101,76,48,180,32.9,0.171,63
764,2,122,70,27,0,36.8,0.340,27
765,5,121,72,23,112,26.2,0.245,30
766,1,126,60,0,0,30.1,0.349,47


In [7]:
y

0      1
1      0
2      1
3      0
4      1
      ..
763    0
764    0
765    0
766    1
767    0
Name: Outcome, Length: 768, dtype: int64

- `Splitting and Stratifying`

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=42, stratify=y)

## 3 - Aplying the algorithm:

### *We'll apply the Logistic Regression, test the differents regularization with different alpha, and see features behavior.*

- `No regularization`

In [137]:
alpha_range = [1000,500,100,10,1,0.1,0.01,0.001]
# higher the alpha value, more restriction on the coefficients
# low alpha > more generalization,
scores_ridge = {'metric':[], 'score':[], 'penalty':[], 'alpha':[]}
coefs = {'penalty':[], 'alpha':[], 'coefs':[]}
penalties = ['l1','l2','elasticnet','none']

for alpha in alpha_range:
    for penalty in penalties:
        if penalty == 'l1':
            model = LogisticRegression(C = 1/alpha, random_state=0, penalty = 'l1', solver="liblinear")
            model.fit(X_train, y_train)
            model_predict = model.predict(X_test)
            
            
        elif penalty == 'elasticnet': # NOT CONVERGES
            model = LogisticRegression(C = 1/alpha, random_state=0, penalty = 'elasticnet', solver="saga", l1_ratio=0.5, max_iter=400)
            model.fit(X_train, y_train)
            model_predict = model.predict(X_test)
            
        elif penalty == 'l2':
            model = LogisticRegression(C = 1/alpha, random_state=0, penalty = 'l2', max_iter=150)
            model.fit(X_train, y_train)
            model_predict = model.predict(X_test)
            
        else: # When we use the penalty like none the alpha is ignored
            model = LogisticRegression(random_state=0, penalty = 'none', max_iter=300)
            model.fit(X_train, y_train)
            model_predict = model.predict(X_test)
            scores_ridge['score'].append(accuracy_score(y_test, model_predict))
            scores_ridge['penalty'].append(penalty)
            scores_ridge['alpha'].append('ignored')
            scores_ridge['metric'].append('accuracy')
            break
        
        
        coefs['penalty'].append(penalty)
        coefs['coefs'].append(model.coef_)
        coefs['alpha'].append(alpha)
        
        
        scores_ridge['score'].append(accuracy_score(y_test, model_predict))
        scores_ridge['penalty'].append(penalty)
        scores_ridge['alpha'].append(alpha)
        scores_ridge['metric'].append('accuracy')
    
scores_ridge = pd.DataFrame.from_dict(scores_ridge)
coefs = pd.DataFrame.from_dict(coefs)

/home/joaocarvoli/miniconda3/envs/ml/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/home/joaocarvoli/miniconda3/envs/ml/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/home/joaocarvoli/miniconda3/envs/ml/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/home/joaocarvoli/miniconda3/envs/ml/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/home/joaocarvoli/miniconda3/envs/ml/lib/pyt

- About the Warning above: [`Logistic regression does cannot converge without poor model performance`](https://datascience.stackexchange.com/questions/77813/logistic-regression-does-cannot-converge-without-poor-model-performance)

In [144]:
scores_ridge.sort_values(by = 'score', ascending=False).head(10)

,metric,score,penalty,alpha
1,accuracy,0.727273,l2,1000
9,accuracy,0.720779,l2,100
5,accuracy,0.720779,l2,500
24,accuracy,0.714286,l1,0.01
15,accuracy,0.714286,none,ignored
20,accuracy,0.714286,l1,0.1
21,accuracy,0.714286,l2,0.1
23,accuracy,0.714286,none,ignored
11,accuracy,0.714286,none,ignored
31,accuracy,0.714286,none,ignored


## *We can see that the in `l2`  penalty and when the alpha = `1000` is where the best result happens.*

In [143]:
for i in range(0,15):
    print('The alpha is: ', coefs.alpha[i], ' and the regularization is: ', coefs.penalty[i])
    print(coefs.coefs[i],'\n')

The alpha is:  1000.0  and the regularization is:  l1
[[ 0.          0.00062052 -0.00892598  0.          0.00128735  0.
   0.          0.        ]] 

The alpha is:  1000.0  and the regularization is:  l2
[[ 0.04865518  0.03610059 -0.00898946  0.00689703 -0.00091027  0.07334893
   0.00740878  0.02188546]] 

The alpha is:  1000.0  and the regularization is:  elasticnet
[[ 0.          0.00620357 -0.01879746  0.          0.00124322  0.
   0.          0.        ]] 

The alpha is:  500.0  and the regularization is:  l1
[[ 0.          0.00627901 -0.01897975  0.          0.00124094  0.
   0.          0.        ]] 

The alpha is:  500.0  and the regularization is:  l2
[[ 0.06666811  0.03637667 -0.01001098  0.00632988 -0.00093793  0.08224839
   0.0143285   0.02084926]] 

The alpha is:  500.0  and the regularization is:  elasticnet
[[ 0.          0.00938317 -0.02446153  0.          0.00121114  0.
   0.          0.        ]] 

The alpha is:  100.0  and the regularization is:  l1
[[ 0.01037494  0.0

## *We can see that how much the alpha increases, more features are discarded, and how much more alpha decreases, the inverse happens.*